In [ ]:
import cv2
import numpy as np
from scipy import stats
from collections import Counter

# Функция для вычисления угла линии
def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

# Функция для вычисления расстояния между двумя точками
def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Функция для вычисления длины линии
def calculate_line_length(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Функция для объединения близких линий
def merge_close_lines(lines, distance_threshold=4, angle_threshold=5):
    """
    Объединяет близко расположенные линии с небольшим угловым отклонением.

    :param lines: Список линий в формате [[x1, y1, x2, y2], ...].
    :param distance_threshold: Максимальное расстояние между линиями для объединения (в пикселях).
    :param angle_threshold: Максимальное угловое отклонение для объединения (в градусах).
    :return: Список объединенных линий.
    """
    merged_lines = []
    used_indices = set()

    for i, line1 in enumerate(lines):
        if i in used_indices:
            continue

        x1, y1, x2, y2 = line1
        angle1 = calculate_angle(x1, y1, x2, y2)
        group = [line1]

        for j, line2 in enumerate(lines[i + 1:], start=i + 1):
            if j in used_indices:
                continue

            x3, y3, x4, y4 = line2
            angle2 = calculate_angle(x3, y3, x4, y4)

            # Проверка углового отклонения
            if abs(angle1 - angle2) > angle_threshold:
                continue

            # Проверка близости линий
            center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
            center2 = ((x3 + x4) / 2, (y3 + y4) / 2)
            distance = calculate_distance(center1, center2)

            if distance > distance_threshold:
                continue

            # Если линии близки и углы схожи, добавляем в группу
            group.append(line2)
            used_indices.add(j)

        # Усреднение линий в группе
        if group:
            avg_line = np.mean(group, axis=0).astype(int)
            merged_lines.append(avg_line)
            used_indices.add(i)

    return merged_lines

# Функция для вычисления среднего угла наклона линий
def calculate_average_angle(lines):
    angles = [calculate_angle(*line) for line in lines]
    return np.mean(angles)

# Функция для поворота изображения на заданный угол с расширением холста
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # Вычисление новых размеров холста
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    
    # Корректировка матрицы поворота для учета нового холста
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    
    # Поворот изображения с расширением холста
    rotated = cv2.warpAffine(image, M, (new_w, new_h), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return rotated

# Функция для определения преобладающего цвета на изображении
def get_dominant_color(image):
    pixels = image.reshape(-1, 3)
    most_common = Counter(map(tuple, pixels)).most_common(1)
    return np.array(most_common[0][0], dtype=np.uint8)

# Функция для масштабирования изображения с заполнением преобладающим цветом
def scale_image(image, target_width, target_height, dominant_color):
    h, w = image.shape[:2]
    scale = min(target_width / w, target_height / h)
    resized = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    
    # Создание холста с преобладающим цветом
    canvas = np.full((target_height, target_width, 3), dominant_color, dtype=np.uint8)
    
    # Размещение изображения по центру холста
    x_offset = (target_width - resized.shape[1]) // 2
    y_offset = (target_height - resized.shape[0]) // 2
    canvas[y_offset:y_offset + resized.shape[0], x_offset:x_offset + resized.shape[1]] = resized
    
    return canvas

# Загрузка изображения
image = cv2.imread('RotateJPG6.jpg', cv2.IMREAD_COLOR)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение Canny edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Применение Hough Line Transform
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)

# Преобразование линий в нужный формат
lines = [line[0] for line in lines]

# Объединение близких линий
merged_lines = merge_close_lines(lines, distance_threshold=4, angle_threshold=5)

# Создание копии изображения для рисования линий
output_image = image.copy()

# Получение высоты и ширины изображения
image_height, image_width, _ = image.shape

# Функция для вычисления центра линии
def calculate_center(x1, y1, x2, y2):
    return ((x1 + x2) // 2, (y1 + y2) // 2)

# Словарь для хранения центров линий
line_centers = {}

# Отрисовка всех линий и сохранение их центров
for i, line in enumerate(merged_lines):
    x1, y1, x2, y2 = line
    center = calculate_center(x1, y1, x2, y2)
    line_centers[i] = center

# Фильтрация линий, центры которых находятся в средней трети ширины изображения
middle_third_start = image_width // 3
middle_third_end = 2 * image_width // 3

# Минимальная длина линии (1/4 ширины изображения)
min_line_length = image_width / 4

# Фильтрация линий по средней трети ширины, длине и углу наклона
filtered_lines = {
    line_id: center for line_id, center in line_centers.items()
    if (middle_third_start <= center[0] <= middle_third_end) and
       (calculate_line_length(*merged_lines[line_id]) >= min_line_length) and
       (-40 <= calculate_angle(*merged_lines[line_id]) <= 40)
}

# Нахождение линий с минимальным и максимальным значением Y-координаты центров
if filtered_lines:
    min_y_line = min(filtered_lines.items(), key=lambda item: item[1][1])
    max_y_line = max(filtered_lines.items(), key=lambda item: item[1][1])

    # Отрисовка линий с минимальным и максимальным Y
    for line_id, center in [min_y_line, max_y_line]:
        x1, y1, x2, y2 = merged_lines[line_id]
        cv2.line(output_image, (x1, y1), (x2, y2), (0, 255, 127), 2)  # Салатовый цвет
        cv2.circle(output_image, center, 5, (0, 255, 255), -1)  # Желтый круг для центра
        cv2.putText(output_image, f"Y={center[1]}", (center[0] + 10, center[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)  # Подпись Y-координаты

    # Вычисление углов наклона для отфильтрованных линий
    angles = [calculate_angle(*merged_lines[line_id]) for line_id in filtered_lines.keys()]

    # Проверка, параллельны ли линии (разница углов не более 2 градусов)
    if np.std(angles) <= 2:
        average_angle = np.mean(angles)
        # Поворот изображения на средний угол (в правильную сторону)
        rotated_image = rotate_image(image, -average_angle)
        
        # Определение преобладающего цвета
        dominant_color = get_dominant_color(image)
        
        # Масштабирование изображения с заполнением преобладающим цветом
        final_image = scale_image(rotated_image, image_width, image_height, dominant_color)
    else:
        final_image = image
else:
    final_image = image

# Сохранение изображения с результатом
cv2.imwrite('output_final.jpg', final_image)

# Показ результата
cv2.imshow('Final Result', final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Функция для вычисления угла линии
def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

# Функция для вычисления расстояния между двумя точками
def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Функция для вычисления длины линии
def calculate_line_length(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Функция для объединения близких линий
def merge_close_lines(lines, distance_threshold=4, angle_threshold=5):
    """
    Объединяет близко расположенные линии с небольшим угловым отклонением.

    :param lines: Список линий в формате [[x1, y1, x2, y2], ...].
    :param distance_threshold: Максимальное расстояние между линиями для объединения (в пикселях).
    :param angle_threshold: Максимальное угловое отклонение для объединения (в градусах).
    :return: Список объединенных линий.
    """
    merged_lines = []
    used_indices = set()

    for i, line1 in enumerate(lines):
        if i in used_indices:
            continue

        x1, y1, x2, y2 = line1
        angle1 = calculate_angle(x1, y1, x2, y2)
        group = [line1]

        for j, line2 in enumerate(lines[i + 1:], start=i + 1):
            if j in used_indices:
                continue

            x3, y3, x4, y4 = line2
            angle2 = calculate_angle(x3, y3, x4, y4)

            # Проверка углового отклонения
            if abs(angle1 - angle2) > angle_threshold:
                continue

            # Проверка близости линий
            center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
            center2 = ((x3 + x4) / 2, (y3 + y4) / 2)
            distance = calculate_distance(center1, center2)

            if distance > distance_threshold:
                continue

            # Если линии близки и углы схожи, добавляем в группу
            group.append(line2)
            used_indices.add(j)

        # Усреднение линий в группе
        if group:
            avg_line = np.mean(group, axis=0).astype(int)
            merged_lines.append(avg_line)
            used_indices.add(i)

    return merged_lines

# Загрузка изображения
image = cv2.imread('Копия E1408970-45B6-4F42-8EED-E36B6D1E7259_non_black.jpg', cv2.IMREAD_COLOR)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение Canny edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Применение Hough Line Transform
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)

# Преобразование линий в нужный формат
lines = [line[0] for line in lines]

# Объединение близких линий
merged_lines = merge_close_lines(lines, distance_threshold=4, angle_threshold=5)

# Создание копии изображения для рисования линий
output_image = image.copy()

# Получение высоты и ширины изображения
image_height, image_width, _ = image.shape

# Функция для вычисления центра линии
def calculate_center(x1, y1, x2, y2):
    return ((x1 + x2) // 2, (y1 + y2) // 2)

# Словарь для хранения центров линий
line_centers = {}

# Отрисовка всех линий и сохранение их центров
for i, line in enumerate(merged_lines):
    x1, y1, x2, y2 = line
    center = calculate_center(x1, y1, x2, y2)
    line_centers[i] = center

# Фильтрация линий, центры которых находятся в средней трети ширины изображения
middle_third_start = image_width // 3
middle_third_end = 2 * image_width // 3

# Минимальная длина линии (1/4 ширины изображения)
min_line_length = image_width / 4

# Фильтрация линий по средней трети ширины, длине и углу наклона
filtered_lines = {
    line_id: center for line_id, center in line_centers.items()
    if (middle_third_start <= center[0] <= middle_third_end) and
       (calculate_line_length(*merged_lines[line_id]) >= min_line_length) and
       (-40 <= calculate_angle(*merged_lines[line_id]) <= 40)
}

# Нахождение линий с минимальным и максимальным значением Y-координаты центров
if filtered_lines:
    min_y_line = min(filtered_lines.items(), key=lambda item: item[1][1])
    max_y_line = max(filtered_lines.items(), key=lambda item: item[1][1])

    # Отрисовка линий с минимальным и максимальным Y
    for line_id, center in [min_y_line, max_y_line]:
        x1, y1, x2, y2 = merged_lines[line_id]
        cv2.line(output_image, (x1, y1), (x2, y2), (0, 255, 127), 2)  # Салатовый цвет
        cv2.circle(output_image, center, 5, (0, 255, 255), -1)  # Желтый круг для центра
        cv2.putText(output_image, f"Y={center[1]}", (center[0] + 10, center[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)  # Подпись Y-координаты

# Сохранение изображения с результатом
cv2.imwrite('output_final.jpg', output_image)

# Показ результата
cv2.imshow('Final Result', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Годный годный годный

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
from collections import Counter
# Загрузка изображения
image_path = 'Rotated4.jpg'
image = Image.open(image_path)

# Вычисление новых размеров с сохранением пропорций
width = 1100
ratio = width / float(image.size[0])
height = int(float(image.size[1]) * ratio)

# Изменение размера изображения с использованием LANCZOS
resized_image = image.resize((width, height), Image.Resampling.LANCZOS)

# Формирование нового имени файла
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
new_filename = f"{name}_resized{ext}"
new_image_path = os.path.join('resized_data/', new_filename)

# Сохранение измененного изображения
resized_image.save(new_image_path)

# Опционально: показ изображения
resized_image.show()

name1=new_image_path
# Функция для вычисления угла линии
def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

# Функция для вычисления расстояния между двумя точками
def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Функция для вычисления длины линии
def calculate_line_length(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Функция для объединения близких линий
def merge_close_lines(lines, distance_threshold=4, angle_threshold=5):
    """
    Объединяет близко расположенные линии с небольшим угловым отклонением.

    :param lines: Список линий в формате [[x1, y1, x2, y2], ...].
    :param distance_threshold: Максимальное расстояние между линиями для объединения (в пикселях).
    :param angle_threshold: Максимальное угловое отклонение для объединения (в градусах).
    :return: Список объединенных линий.
    """
    merged_lines = []
    used_indices = set()

    for i, line1 in enumerate(lines):
        if i in used_indices:
            continue

        x1, y1, x2, y2 = line1
        angle1 = calculate_angle(x1, y1, x2, y2)
        group = [line1]

        for j, line2 in enumerate(lines[i + 1:], start=i + 1):
            if j in used_indices:
                continue

            x3, y3, x4, y4 = line2
            angle2 = calculate_angle(x3, y3, x4, y4)

            # Проверка углового отклонения
            if abs(angle1 - angle2) > angle_threshold:
                continue

            # Проверка близости линий
            center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
            center2 = ((x3 + x4) / 2, (y3 + y4) / 2)
            distance = calculate_distance(center1, center2)

            if distance > distance_threshold:
                continue

            # Если линии близки и углы схожи, добавляем в группу
            group.append(line2)
            used_indices.add(j)

        # Усреднение линий в группе
        if group:
            avg_line = np.mean(group, axis=0).astype(int)
            merged_lines.append(avg_line)
            used_indices.add(i)

    return merged_lines

# Загрузка изображения
image = cv2.imread(name1, cv2.IMREAD_COLOR)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение Canny edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Применение Hough Line Transform
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)

# Преобразование линий в нужный формат
lines = [line[0] for line in lines]

# Объединение близких линий
merged_lines = merge_close_lines(lines, distance_threshold=4, angle_threshold=5)

# Создание копии изображения для рисования линий
output_image = image.copy()

# Получение высоты и ширины изображения
image_height, image_width, _ = image.shape

# Функция для вычисления центра линии
def calculate_center(x1, y1, x2, y2):
    return ((x1 + x2) // 2, (y1 + y2) // 2)

# Словарь для хранения центров линий
line_centers = {}

# Отрисовка всех линий и сохранение их центров
for i, line in enumerate(merged_lines):
    x1, y1, x2, y2 = line
    center = calculate_center(x1, y1, x2, y2)
    line_centers[i] = center

# Фильтрация линий, центры которых находятся в средней трети ширины изображения
middle_third_start = image_width // 3
middle_third_end = 2 * image_width // 3

# Минимальная длина линии (1/4 ширины изображения)
min_line_length = image_width / 4

# Фильтрация линий по средней трети ширины, длине и углу наклона
filtered_lines = {
    line_id: center for line_id, center in line_centers.items()
    if (middle_third_start <= center[0] <= middle_third_end) and
       (calculate_line_length(*merged_lines[line_id]) >= min_line_length) and
       (-40 <= calculate_angle(*merged_lines[line_id]) <= 40)
}

# Нахождение линий с минимальным и максимальным значением Y-координаты центров
if filtered_lines:
    min_y_line = min(filtered_lines.items(), key=lambda item: item[1][1])
    max_y_line = max(filtered_lines.items(), key=lambda item: item[1][1])

    # Отрисовка линий с минимальным и максимальным Y
    for line_id, center in [min_y_line, max_y_line]:
        x1, y1, x2, y2 = merged_lines[line_id]
        angle = calculate_angle(x1, y1, x2, y2)
        cv2.line(output_image, (x1, y1), (x2, y2), (0, 255, 127), 2)  # Салатовый цвет
        cv2.circle(output_image, center, 5, (0, 255, 255), -1)  # Желтый круг для центра
        cv2.putText(output_image, f"Y={center[1]}", (center[0] + 10, center[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)  # Подпись Y-координаты
        cv2.putText(output_image, f"Angle={angle:.2f}", (center[0] + 10, center[1] + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)  # Подпись угла наклона

# Формирование нового имени файла
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
new_filename = f"{name}_int_res{ext}"
new_image_path1 = os.path.join('intermediate_results/', new_filename)

# Сохранение изображения с результатом
cv2.imwrite(new_image_path1, output_image)

# Показ результата
cv2.imshow('Final Result', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
def get_dominant_color(image_path, num_colors=1):
    # Открываем изображение
    image = Image.open(image_path)

    # Уменьшаем размер изображения для ускорения обработки
    image = image.resize((150, 150))

    # Преобразуем изображение в массив numpy
    image_array = np.array(image)

    # Преобразуем массив в список пикселей
    pixels = image_array.reshape(-1, image_array.shape[-1])

    # Удаляем альфа-канал, если он есть
    if pixels.shape[1] == 4:
        pixels = pixels[:, :3]

    # Подсчитываем количество уникальных цветов
    color_counts = Counter(map(tuple, pixels))

    # Находим наиболее часто встречающийся цвет
    dominant_color = color_counts.most_common(num_colors)[0][0]

    return dominant_color

# Путь к вашему изображению
image_path = name

# Загрузка изображения с помощью OpenCV
image = cv2.imread(image_path)

# Получаем доминирующий цвет
dominant_color = get_dominant_color(image_path)

# Выводим результат
print(f"Доминирующий цвет: {dominant_color}")

# Проверка, что найдены две линии
if len(filtered_lines) >= 2:
    # Получаем углы наклона для двух линий
    angle1 = calculate_angle(*merged_lines[min_y_line[0]])
    angle2 = calculate_angle(*merged_lines[max_y_line[0]])

    # Вычисляем средний угол
    average_angle = (angle1 + angle2) / 2
# Преобразуем dominant_color в кортеж (B, G, R)
if isinstance(dominant_color, (tuple, list, np.ndarray)):
    # Если dominant_color уже является кортежем, списком или массивом, преобразуем его в кортеж целых чисел
    dominant_color = tuple(map(int, dominant_color))
else:
    # Если dominant_color одно число (например, градация серого), дублируем его три раза
    dominant_color = (int(dominant_color),) * 3

# Вычисляем матрицу поворота
(h, w) = image.shape[:2]
center = (w // 2, h // 2)
rotation_matrix = cv2.getRotationMatrix2D(center, average_angle, 1.0)

# Вычисляем новые размеры холста после поворота
cos = np.abs(rotation_matrix[0, 0])
sin = np.abs(rotation_matrix[0, 1])
new_w = int((h * sin) + (w * cos))
new_h = int((h * cos) + (w * sin))

# Корректируем матрицу поворота для учета нового холста
rotation_matrix[0, 2] += (new_w / 2) - center[0]
rotation_matrix[1, 2] += (new_h / 2) - center[1]

# Применяем поворот к изображению с новыми размерами холста
# Указываем borderValue для заполнения новых областей доминирующим цветом
rotated_image = cv2.warpAffine(
    image,
    rotation_matrix,
    (new_w, new_h),
    flags=cv2.INTER_LINEAR,
    borderMode=cv2.BORDER_CONSTANT,
    borderValue=dominant_color  # Заполняем новые области доминирующим цветом
)

# Формирование нового имени файла
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
new_filename = f"{name}_rotated_image{ext}"
new_image_path = os.path.join('Output_data/', new_filename)

# Сохранение результата
cv2.imwrite(new_image_path, rotated_image)

# Показ результата
cv2.imshow('rotated_image', rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Исправление поворота завершено. Результат сохранен в файл 'rotated_output.jpg'.")

In [ ]:
import os
import shutil

# Функция для проверки параллельности линий
def are_lines_parallel(angle1, angle2, angle_threshold=2):
    return abs(angle1 - angle2) <= angle_threshold

# Проверка условий для перемещения файла
if len(filtered_lines) < 2 or not are_lines_parallel(angle1, angle2, angle_threshold=1):
    # Создаем папку, если она не существует
    output_folder = 'non_parallel_lines'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Перемещаем файл в папку
    shutil.move(name, os.path.join(output_folder, name))
    print(f"Файл '{name}' перемещен в папку '{output_folder}'.")
else:
    print("Линии параллельны, файл остается в исходной папке.")

Годный пакетный

In [ ]:
import cv2
import numpy as np
from PIL import Image
from collections import Counter
import os

# Функция для вычисления угла линии
def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

# Функция для вычисления расстояния между двумя точками
def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Функция для вычисления длины линии
def calculate_line_length(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Функция для объединения близких линий
def merge_close_lines(lines, distance_threshold=4, angle_threshold=5):
    merged_lines = []
    used_indices = set()

    for i, line1 in enumerate(lines):
        if i in used_indices:
            continue

        x1, y1, x2, y2 = line1
        angle1 = calculate_angle(x1, y1, x2, y2)
        group = [line1]

        for j, line2 in enumerate(lines[i + 1:], start=i + 1):
            if j in used_indices:
                continue

            x3, y3, x4, y4 = line2
            angle2 = calculate_angle(x3, y3, x4, y4)

            if abs(angle1 - angle2) > angle_threshold:
                continue

            center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
            center2 = ((x3 + x4) / 2, (y3 + y4) / 2)
            distance = calculate_distance(center1, center2)

            if distance > distance_threshold:
                continue

            group.append(line2)
            used_indices.add(j)

        if group:
            avg_line = np.mean(group, axis=0).astype(int)
            merged_lines.append(avg_line)
            used_indices.add(i)

    return merged_lines

# Функция для получения доминирующего цвета
def get_dominant_color(image_path, num_colors=1):
    image = Image.open(image_path)
    image = image.resize((150, 150))
    image_array = np.array(image)
    pixels = image_array.reshape(-1, image_array.shape[-1])

    if pixels.shape[1] == 4:
        pixels = pixels[:, :3]

    color_counts = Counter(map(tuple, pixels))
    dominant_color = color_counts.most_common(num_colors)[0][0]
    return dominant_color

# Папки для обработки
input_folder = 'input_images'
intermediate_folder = 'intermediate_results'
output_folder = 'final_results'

# Создаем папки, если они не существуют
os.makedirs(intermediate_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Обработка всех изображений в папке
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Промежуточный результат: поиск линий и углов
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)
        lines = [line[0] for line in lines]
        merged_lines = merge_close_lines(lines, distance_threshold=4, angle_threshold=5)

        output_image = image.copy()
        image_height, image_width, _ = image.shape

        line_centers = {}
        for i, line in enumerate(merged_lines):
            x1, y1, x2, y2 = line
            center = ((x1 + x2) // 2, (y1 + y2) // 2)
            line_centers[i] = center

        middle_third_start = image_width // 3
        middle_third_end = 2 * image_width // 3
        min_line_length = image_width / 4

        filtered_lines = {
            line_id: center for line_id, center in line_centers.items()
            if (middle_third_start <= center[0] <= middle_third_end) and
               (calculate_line_length(*merged_lines[line_id]) >= min_line_length) and
               (-40 <= calculate_angle(*merged_lines[line_id]) <= 40)
        }

        if filtered_lines:
            min_y_line = min(filtered_lines.items(), key=lambda item: item[1][1])
            max_y_line = max(filtered_lines.items(), key=lambda item: item[1][1])

            for line_id, center in [min_y_line, max_y_line]:
                x1, y1, x2, y2 = merged_lines[line_id]
                angle = calculate_angle(x1, y1, x2, y2)
                cv2.line(output_image, (x1, y1), (x2, y2), (0, 255, 127), 2)
                cv2.circle(output_image, center, 5, (0, 255, 255), -1)
                cv2.putText(output_image, f"Y={center[1]}", (center[0] + 10, center[1]),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
                cv2.putText(output_image, f"Angle={angle:.2f}", (center[0] + 10, center[1] + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

        # Сохранение промежуточного результата
        intermediate_output_path = os.path.join(intermediate_folder, f'intermediate_{filename}')
        cv2.imwrite(intermediate_output_path, output_image)

        # Окончательное преобразование: поворот и заполнение фона
        dominant_color = get_dominant_color(image_path)
        if filtered_lines:
            angle1 = calculate_angle(*merged_lines[min_y_line[0]])
            angle2 = calculate_angle(*merged_lines[max_y_line[0]])
            average_angle = (angle1 + angle2) / 2

            h, w = image.shape[:2]
            center = (w // 2, h // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, average_angle, 1.0)

            cos = np.abs(rotation_matrix[0, 0])
            sin = np.abs(rotation_matrix[0, 1])
            new_w = int((h * sin) + (w * cos))
            new_h = int((h * cos) + (w * sin))

            rotation_matrix[0, 2] += (new_w / 2) - center[0]
            rotation_matrix[1, 2] += (new_h / 2) - center[1]

            rotated_image = cv2.warpAffine(image, rotation_matrix, (new_w, new_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

            mask = np.all(rotated_image == [0, 0, 0], axis=-1)
            kernel = np.ones((5, 5), np.uint8)
            mask_closed = cv2.morphologyEx(mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
            mask_dilated = cv2.dilate(mask_closed, kernel, iterations=5)
            rotated_image[mask_dilated == 1] = dominant_color

            # Сохранение окончательного результата
            final_output_path = os.path.join(output_folder, f'final_{filename}')
            cv2.imwrite(final_output_path, rotated_image)

print("Обработка всех изображений завершена.")

Последний работает

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
from collections import Counter

# Загрузка изображения
image_path = 'Input_data/RotateJPG6.jpg'
image = Image.open(image_path)

# Вычисление новых размеров с сохранением пропорций
width = 1100
ratio = width / float(image.size[0])
height = int(float(image.size[1]) * ratio)

# Изменение размера изображения с использованием LANCZOS
resized_image = image.resize((width, height), Image.Resampling.LANCZOS)

# Формирование нового имени файла
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
new_filename = f"{name}_resized{ext}"
new_image_path = os.path.join('resized_data/', new_filename)

# Сохранение измененного изображения
resized_image.save(new_image_path)

# Опционально: показ изображения
resized_image.show()

name1 = new_image_path

# Функция для вычисления угла линии
def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

# Функция для вычисления расстояния между двумя точками
def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Функция для вычисления длины линии
def calculate_line_length(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Функция для объединения близких линий
def merge_close_lines(lines, distance_threshold=4, angle_threshold=5):
    merged_lines = []
    used_indices = set()

    for i, line1 in enumerate(lines):
        if i in used_indices:
            continue

        x1, y1, x2, y2 = line1
        angle1 = calculate_angle(x1, y1, x2, y2)
        group = [line1]

        for j, line2 in enumerate(lines[i + 1:], start=i + 1):
            if j in used_indices:
                continue

            x3, y3, x4, y4 = line2
            angle2 = calculate_angle(x3, y3, x4, y4)

            if abs(angle1 - angle2) > angle_threshold:
                continue

            center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
            center2 = ((x3 + x4) / 2, (y3 + y4) / 2)
            distance = calculate_distance(center1, center2)

            if distance > distance_threshold:
                continue

            group.append(line2)
            used_indices.add(j)

        if group:
            avg_line = np.mean(group, axis=0).astype(int)
            merged_lines.append(avg_line)
            used_indices.add(i)

    return merged_lines

# Загрузка изображения
image = cv2.imread(name1, cv2.IMREAD_COLOR)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение Canny edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Применение Hough Line Transform
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)

# Преобразование линий в нужный формат
lines = [line[0] for line in lines]

# Объединение близких линий
merged_lines = merge_close_lines(lines, distance_threshold=4, angle_threshold=5)

# Создание копии изображения для рисования линий
output_image = image.copy()

# Получение высоты и ширины изображения
image_height, image_width, _ = image.shape

# Функция для вычисления центра линии
def calculate_center(x1, y1, x2, y2):
    return ((x1 + x2) // 2, (y1 + y2) // 2)

# Словарь для хранения центров линий
line_centers = {}

# Отрисовка всех линий и сохранение их центров
for i, line in enumerate(merged_lines):
    x1, y1, x2, y2 = line
    center = calculate_center(x1, y1, x2, y2)
    line_centers[i] = center

# Выбор любых двух линий для вычисления среднего угла
if len(merged_lines) >= 2:
    line1 = merged_lines[0]
    line2 = merged_lines[1]

    angle1 = calculate_angle(*line1)
    angle2 = calculate_angle(*line2)
    average_angle = (angle1 + angle2) / 2
else:
    # Если не найдено две линии, используем угол по умолчанию (например, 0 градусов)
    average_angle = 0

# Получаем доминирующий цвет
def get_dominant_color(image_path, num_colors=1):
    image = Image.open(image_path)
    image = image.resize((150, 150))
    image_array = np.array(image)
    pixels = image_array.reshape(-1, image_array.shape[-1])
    if pixels.shape[1] == 4:
        pixels = pixels[:, :3]
    color_counts = Counter(map(tuple, pixels))
    dominant_color = color_counts.most_common(num_colors)[0][0]
    return dominant_color

dominant_color = get_dominant_color(image_path)

# Преобразуем dominant_color в кортеж (B, G, R)
if isinstance(dominant_color, (tuple, list, np.ndarray)):
    dominant_color = tuple(map(int, dominant_color))
else:
    dominant_color = (int(dominant_color),) * 3

# Вычисляем матрицу поворота
(h, w) = image.shape[:2]
center = (w // 2, h // 2)
rotation_matrix = cv2.getRotationMatrix2D(center, average_angle, 1.0)

# Вычисляем новые размеры холста после поворота
cos = np.abs(rotation_matrix[0, 0])
sin = np.abs(rotation_matrix[0, 1])
new_w = int((h * sin) + (w * cos))
new_h = int((h * cos) + (w * sin))

# Корректируем матрицу поворота для учета нового холста
rotation_matrix[0, 2] += (new_w / 2) - center[0]
rotation_matrix[1, 2] += (new_h / 2) - center[1]

# Применяем поворот к изображению с новыми размерами холста
rotated_image = cv2.warpAffine(
    image,
    rotation_matrix,
    (new_w, new_h),
    flags=cv2.INTER_LINEAR,
    borderMode=cv2.BORDER_CONSTANT,
    borderValue=dominant_color
)

# Формирование нового имени файла
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
new_filename = f"{name}_rotated_image{ext}"
new_image_path = os.path.join('Output_data/', new_filename)

# Сохранение результата
cv2.imwrite(new_image_path, rotated_image)

# Показ результата
cv2.imshow('rotated_image', rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Исправление поворота завершено. Результат сохранен в файл 'rotated_output.jpg'.")